<a href="https://colab.research.google.com/github/chaeun6-cmd/New-repository/blob/main/02_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.formula.api as smf
import statsmodels.api as sm

from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, roc_auc_score
)
from sklearn.model_selection import cross_val_score, KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.inspection import permutation_importance, partial_dependence

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_excel("/Online_Retail.xlsx")

df.info()
df.isnull().any()
df.head()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [ ]:
# 고객 결측 제거
df = df[df["CustomerID"].notna()]

# 수량/가격 필터링
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

# 문자열 변환 (에러 방지)
df["StockCode"] = df["StockCode"].astype(str)
df["Description"] = df["Description"].astype(str)

# 비상품(서비스 항목-배송비, 운송비, 은행 수수료, 온라인 배송비, 샘플 상품) 키워드 정의
service_keywords = ["POST", "CARRIAGE", "BANK", "DOTCOM", "SAMPLE"]

pattern = "|".join(service_keywords)

# 비상품 제거
data = df[
    ~df["StockCode"].str.contains(pattern, case=False, na=False) &
    ~df["Description"].str.contains(pattern, case=False, na=False)
]

In [ ]:
# 매출 컬럼  생성
data["TotalAmount"] = data["Quantity"] * data["UnitPrice"]

In [ ]:
# 상품 데이터 정형화(문자열 변환)
data["Description"] = data["Description"].astype(str)

# 대문자 통일
data["Description"] = data["Description"].str.upper()

# 앞뒤 공백 제거
data["Description"] = data["Description"].str.strip()

# 여러 공백을 하나로
data["Description"] = data["Description"].str.replace(r"\s+", " ", regex=True)

In [ ]:
# 매출 컬럼 확인
data["TotalAmount"].describe()

,TotalAmount
count,394780.000000
mean,22.256797
std,309.918923
min,0.001000
25%,4.680000
50%,11.800000
75%,19.800000
max,168469.600000


In [ ]:
# 중복행 제거 (매출 왜곡 방지)
data.duplicated().sum()

np.int64(5172)

In [ ]:
# 기준일 생성(최근성을 계산하기 위해)
reference_date = data["InvoiceDate"].max() + pd.Timedelta(days=1)

In [ ]:
# 가장 최근 구매 날짜/몇 번 주문했는지/총 구매 금액
rfm = data.groupby("CustomerID").agg({
    "InvoiceDate": "max",
    "InvoiceNo": "nunique",
    "TotalAmount": "sum"
}).reset_index()

In [ ]:
# 생성된 기준일을 기준으로 계산
rfm["Recency"] = (reference_date - rfm["InvoiceDate"]).dt.days

In [ ]:
# 컬럼 이름 변환
rfm = rfm.rename(columns={
    "InvoiceNo": "Frequency",
    "TotalAmount": "Monetary"
})

In [ ]:
rfm.head()

,CustomerID,InvoiceDate,Frequency,Monetary,Recency
0,12346.0,2011-01-18 10:01:00,1,77183.60,326
1,12347.0,2011-12-07 15:52:00,7,4310.00,2
2,12348.0,2011-09-25 13:13:00,4,1437.24,75
3,12349.0,2011-11-21 09:51:00,1,1457.55,19
4,12350.0,2011-02-02 16:01:00,1,294.40,310


In [ ]:
rfm.tail()

,CustomerID,InvoiceDate,Frequency,Monetary,Recency
4332,18280.0,2011-03-07 09:52:00,1,180.60,278
4333,18281.0,2011-06-12 10:53:00,1,80.82,181
4334,18282.0,2011-12-02 11:43:00,2,178.05,8
4335,18283.0,2011-12-06 12:02:00,16,2090.53,4
4336,18287.0,2011-10-28 09:29:00,3,1819.88,43


#쿠폰 마진률 지표 만들기

In [ ]:
# 매출 만들기 (총매출 = 구매 수량 x 상품 1개당 가격)
df["TotalAmount"] = df["Quantity"] * df["UnitPrice"]

In [ ]:
# 0원 상품 제거(샘플, 사은품 등)
df = df[df["UnitPrice"] > 0]

# 비회원 거래 제거
df = df[df["CustomerID"].notnull()]

In [ ]:
# 고객 전체 평균 주문금액 계산
invoice_df["TotalAmount"].mean()

np.float64(480.08877836440035)

In [ ]:
# 국가별 거래 수 확인
df["Country"].value_counts()
# 국가별 거래 수 비율로 확
df["Country"].value_counts(normalize=True) * 100

,proportion
Country,
United Kingdom,89.051332
Germany,2.272019
France,2.096340
EIRE,1.818621
Spain,0.624303
Netherlands,0.592886
Belgium,0.510450
Switzerland,0.462698
Portugal,0.367444


In [ ]:
# 주문 단위 집계(주문 번호, 고객ID, 구매 날짜)- 주문1건에 1번 적용(쿠폰 적용 시 순수익 계산)
invoice_df = df.groupby(
    ["InvoiceNo", "CustomerID", "InvoiceDate"],
    as_index=False
)["TotalAmount"].sum()

| 세그먼트   | 쿠폰        |
| ------ | --------- |
| 우수 고객  | £30 (10%) |
| 일반 고객  | £60 (20%) |
| 이탈 위험군 | £90 (30%) |
| 신규 고객  | £60 (20%) |


우수고객은 쿠폰이 없어도 살 확률이 높다고 생각 -> 불필요한 할인 비용(마진 깍일 수 있음)

반대로 이탈 위험군은 최근 구매X, 재구매 가능성도 낮기에 쿠폰 때문에 살 가능성이 증가




In [ ]:
purchase_df = invoice_df[invoice_df["TotalAmount"] > 0].copy()

# 기준일(데이터 마지막 날짜 다음날로 두면 깔끔)
snapshot_date = purchase_df["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = purchase_df.groupby("CustomerID").agg(
    Recency=("InvoiceDate", lambda x: (snapshot_date - x.max()).days),
    Frequency=("InvoiceNo", "nunique"),
    Monetary=("TotalAmount", "sum")
).reset_index()

In [ ]:
# 분위수 기준 컷
r_q1, r_q3 = rfm["Recency"].quantile([0.25, 0.75])
f_q3 = rfm["Frequency"].quantile(0.75)

def segment(row):
    if row["Recency"] <= r_q1 and row["Frequency"] >= f_q3:
        return "우수고객"
    if row["Recency"] >= r_q3:
        return "이탈위험"
    if row["Frequency"] == 1 and row["Recency"] <= r_q1:
        return "신규고객"
    return "일반"

rfm["Segment"] = rfm.apply(segment, axis=1)

In [ ]:
# 주문금액 중앙값 기준(20%) 계산
median_order = invoice_df.loc[invoice_df["TotalAmount"] > 0, "TotalAmount"].median()
base_coupon = 0.20 * median_order

# 우수=0.5배, 일반=1배, 신규=1배, 이탈위험=1.5배 세그먼트별 쿠폰 배수 설정
coupon_map = {
    "우수고객": 0.5 * base_coupon,
    "일반":    1.0 * base_coupon,
    "신규고객": 1.0 * base_coupon,
    "이탈위험": 1.5 * base_coupon
}
# 실제로 고객별 쿠폰 할당
rfm["CouponValue"] = rfm["Segment"].map(coupon_map)

In [ ]:
# 고객별 전체 매출 계산
customer_sales = invoice_df.groupby("CustomerID", as_index=False)["TotalAmount"].sum()
customer_sales = customer_sales.rename(columns={"TotalAmount": "Sales"})

In [ ]:
# 마진율 설정(추정 마진 = 매출 * 0.3)
margin_rate = 0.3

# 고객 매출과 세그먼트 한 테이블에 모으기
result = rfm.merge(customer_sales, on="CustomerID", how="left")

# 쿠폰 비용은 "고객 1명당 쿠폰 1장" 가정/ 쿠폰 비용(=쿠폰 금액) 설정
result["CouponCost"] = result["CouponValue"]

# 쿠폰 적용 후 매출(순매출 = 원래 매출 - 쿠폰)
result["NetSales"] = result["Sales"] - result["CouponCost"]

# 쿠폰 적용 후 순이익(추정)
result["Profit"] = (result["Sales"] * margin_rate) - result["CouponCost"]

# 쿠폰 마진율 계산 (쿠폰 마진율 = 쿠폰적용매출/순이익)
result["CouponMarginRate"] = result["Profit"] / result["NetSales"]

# 0 이하 방지 처리(마진율 왜곡될까 싶어서)
result.loc[result["NetSales"] <= 0, "CouponMarginRate"] = None

##세그먼트별 최종 성과 요약표

In [ ]:
# 고객을 세그먼트별로 묶기
segment_kpi = result.groupby("Segment").agg(
    고객수=("CustomerID", "nunique"), # 해당 세그먼트 고객 유무
    평균매출=("Sales", "mean"),       #고객들의 평균 총 매출
    평균쿠폰=("CouponValue", "mean"), # 세그먼트 고객들에게 지급한 평균 쿠폰 금액
    평균순이익=("Profit", "mean"),   # 평균적으로 순이익이 어떻게 되는지
    평균쿠폰마진율=("CouponMarginRate", "mean") # 세그먼트 전체 평균 쿠폰 마진율
).reset_index()
segment_kpi

,Segment,고객수,평균매출,평균쿠폰,평균순이익,평균쿠폰마진율
0,신규고객,103,316.566117,60.5750,34.394835,-0.098719
1,우수고객,623,7429.900594,30.2875,2198.682678,0.291119
2,이탈위험,1088,651.676756,90.8625,104.640527,-0.525702
3,일반,2524,1402.911733,60.5750,360.298520,-0.001265


##세그먼트별 결과 해석
평균쿠폰마진율  음수인 이유 -> 쿠폰이 매출 대비 상대적으로 크다.

1. 신규 고객
- 쿠폰이 매출 대비 상대적으로 큼
-> 쿠폰율을 줄이거나 최소 구매 조건 필요

2. 우수고객
- 매출이 크고 쿠폰이 작아서 마진율이 플러스

3. 이탈위험
- 쿠폰 대비 매출이 작음
-> 쿠폰율 낮추거나 최소구매 조건 필요

4. 일반 고객
- 거의 손익분기점